In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path(sys.path[0]).parent))
sys.path.append('/home/george/ml/bertenglish/bertviz')
from src.ingredient_tokenizer import load_tokenizer
from transformers import pipeline
from transformers import RobertaForMaskedLM
from bertviz import head_view
import numpy as np
tokenizer = load_tokenizer('../artifacts')

model = RobertaForMaskedLM.from_pretrained('../artifacts', output_attentions=True)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at ../artifacts and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
def show_head_view(model, tokenizer, text):
    inputs = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    attention = model(input_ids)[-1]
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    head_view(attention, tokens)

In [3]:
def get_attention(texts, model, tokenizer):
    inputs = tokenizer.batch_encode_plus(texts, return_tensors='pt', add_special_tokens=True)
    input_ids = inputs['input_ids']
    attention = model(input_ids)[-1]
    return np.stack([a.detach().numpy() for a in attention])

In [4]:
texts = ["salt sugar baking_soda lemon_juice flour egg shortening",
         "salt sugar baking_soda flour molasses egg shortening",
         "salt sugar baking_soda flour egg brown_sugar shortening",
         "salt sugar baking_soda flour egg shortening buttermilk",
        ]
attention = get_attention(texts, model, tokenizer)

In [10]:
show_head_view(model, tokenizer, texts[1])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [485]:
texts = ["salad vinegar olive_oil minced_beef",
         "salad olive_oil apple_cider_vinegar minced_beef",
         "salad red_wine_vinegar vegetable_oil minced_beef"
        ]
attention = get_attention(texts, model, tokenizer)

In [486]:
attention.shape

(8, 3, 12, 6, 6)

In [487]:
attention.argmax(4).shape

(8, 3, 12, 6)

In [489]:
layer= 5
attention.argmax(4)[layer, :, :, [2,3,4]]

array([[[3, 0, 1, 5, 5, 5, 0, 4, 5, 5, 3, 5],
        [1, 4, 3, 5, 2, 5, 4, 0, 1, 3, 3, 1],
        [5, 4, 1, 3, 5, 1, 4, 2, 5, 4, 4, 4]],

       [[1, 4, 4, 5, 5, 5, 4, 0, 5, 2, 0, 0],
        [1, 4, 3, 1, 5, 1, 1, 4, 2, 3, 4, 0],
        [1, 4, 2, 3, 3, 3, 5, 0, 5, 5, 4, 5]],

       [[1, 2, 2, 3, 3, 3, 4, 4, 2, 1, 4, 2],
        [3, 4, 3, 2, 2, 3, 4, 4, 1, 1, 4, 4],
        [1, 4, 2, 2, 1, 3, 4, 2, 1, 1, 4, 4]]])

In [431]:
recipe ="lemon_zest parsley salt asparagus lemons juice pepper salad vegetable_oil  sugar tomatoes shrimp"


attention = get_attention([recipe], model, tokenizer)

tokens = tokenizer.tokenize(recipe)
ingredients = np.argwhere(attention.argmax(4)[2, 0, 7, :] == 1).flatten()#.item()


print([tokens[item-1] for item in ingredients])

[]


In [429]:
!cat ../data/processed/recipes_train.txt | grep 'salad '

herbs plums peach garlic_powder escarole salad sugar cider_vinegar season parsley celery_seeds chicken_breasts pecan salt radicchio honey basil ground_black_pepper chives ground_coriander yellow_mustard_seeds canola_oil red_onion crumbled_blue_cheese
green_onion olive_oil kosher_salt lemon_juice salad apple_cider_vinegar cayenne_pepper smoked_paprika dressing parsley ketchup romaine_lettuce worcestershire_sauce cherry_tomatoes medium_shrimp eggs mayonnaise avocado brown_sugar creme_fraiche
salt bacon dry_mustard apple_cider_vinegar poppy_seed vegetable_oil sugar red_onion swiss_cheese lettuce spinach salad button_mushrooms
butter green_onions olive_oil lime white_kidney sweet_onion chicken_broth minced_garlic cilantro cannellini_beans chili_seasoning salad monterey_jack_cheese heavy_whipping_cream chick_peas skin
chili_sauce ginger salt pineapple_juice mirin garlic_cloves ground_black_pepper red_chile soy_sauce minced_onion sugar sesame_oil lettuce scallions asian_pear salad rice_wine


limes thai_fish_sauce carrot mint_leaves salad chile_paste red_cabbage basil_leaves flank_steak dressing garlic cooking_spray ginger cracked_black_pepper peanuts watercress cilantro_leaves brown_sugar lime_juice
ground_chipotle_chile_pepper black_beans onion avocado spring boneless_skinless_chicken_breast_halves salad mango dressing
pineapple_chunks iceberg_lettuce buttermilk sweet_corn limes parsley_leaves garlic_chives salad celery spinach cucumbers pepper red_onion salt_water frozen_peas grape_tomatoes dried_cranberries cloves frozen_corn
olive_oil minced_parsley parsley_leaves cream chicken_meat carrot egg puff_pastry salad liquid dry_sherry dry_white_wine frozen_corn water minced_onion beef_stock unsalted_butter cornstarch minced_garlic pear_tomatoes frozen_peas freshly_ground_pepper salt baby_greens dried_thyme cooked_chicken dried_rosemary sliced_mushrooms madeira
vinegar papaya honey dry_mustard oil poppy_seed grapes white_wine salad spinach
celery potato parsley light_mayonnai

lettuce black_olives chickpeas sirloin_steak red_capsicum salad red_onion dressing
salt pistachios milk crystallized_ginger mango_puree oil mangoes water salad sugar asafetida lime_juice
paprika eggs salt light_mayonnaise onion celery_ribs garnish pickle_relish sweet_pepper orange celery_seed macaroni sugar white_vinegar dill_pickle yellow_mustard salad apple_cider_vinegar
pumpkin_seeds red_chili_powder ground salad star_anise corn_kernels season sweetener chipotle_powder red_wine_vinegar pepitas grapeseed_oil salt honey spring_greens dijon_mustard strawberries turnips canola_oil romaine_lettuce_hearts
green_bell_pepper black_pepper sliced_green_onions avocados salad guacamole vegetable_oil garlic_minced flour_tortillas flank_steak seasoning romaine_lettuce sour_cream roma_tomatoes salt yellow_bell_pepper lime_wedges salsa jalapeno_pepper
pork_chops flour salt olive_oil vinegar ground_black_pepper arugula plum_tomatoes vegetable_oil red_onion egg loin mozzarella_cheese salad dry_bread_

carrot fruits season salad spinach
olive_oil sauce carrots mortadella rosemary_leaves dried_porcini_mushrooms salad ground_nutmeg celery cooked_pasta dry_white_wine chicken_stock spinach garlic onion pork_butt boneless_pork eggs salt ground_black_pepper bechamel cloves
fillets flour bouillon_granules onion cucumber vinegar bacon black_pepper chives capers oil potatoes mustard_seeds ground_mustard sole radish salad vegetable_bouillon
spinach_leaves parsley salmon lime ground_black_pepper sprig butternut_squash ground_cumin jalapeno_pepper salad vinaigrette_dressing
parsley mayonnaise garlic_salt avocado toasted_sesame_seeds red_salmon dill_weed lemon_juice salad fat
cannellini olive_oil dijon_mustard white_kidney_beans beets salt_and_ground_black_pepper baby_arugula sugar salad walnuts cider_vinegar crumbled_blue_cheese dressing
black_pepper peas parsley_leaves carrot puff_pastry salad liquid dry_sherry thyme rosemary dry_white_wine water extra_virgin_olive_oil onion garlic_salt beef_st

butter celery flour salt onion milk chicken_broth pepper salad carrot egg frozen_peas frozen_corn
garlic butter salt green_onion basil feta_cheese_crumbles tomato naan artichoke_hearts salad shrimp
green_onion black_pepper rounds garlic_powder lemon_juice asian hot_sauce salad celery crabmeat pasta brown rice_paper angel_hair sriracha soy_sauce rice_noodles filling mayonnaise
red_chili chili_sauce minced_ginger chicken_breast baby_corn cucumber mint_leaves cilantro soy_sauce salad lime_juice dressing red_onion sliced_carrots rice_noodles dry_roasted_peanuts
apple orange blue_cheese walnuts salad vinaigrette_dressing
tofu fish_sauce eggs bean_sprouts green_onions boiling_water minced_garlic rice_wine_vinegar powder soy_sauce peanut_oil lime_juice brown_sugar kimchi salad shrimp
tomato carrots lemon_juice salad parsley_sprigs celery thyme rosemary extra_virgin_olive_oil anchovy onion cold_water roast capers salt mayonnaise basil dijon_mustard bay_leaves ground_pepper breast
greens olive_

In [418]:
item

array([1, 2, 3])

In [406]:
attention.argmax(4)[0, :, 1, :]

array([[4, 1, 1, 1, 0]])

In [289]:
model_version = 'bert-base-uncased'
do_lower_case = True
model = AutoModel.from_pretrained(model_version, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)

In [366]:
texts = ["take a bath",
"eat a burger",
"walk the dog",
"kick the ball",
'the cat sat']

attention = get_attention(texts, model, tokenizer)

In [367]:
layer = 9
attention.argmax(-2)[layer, :, :, 1]

array([[1, 2, 3, 3, 0, 4, 1, 1, 0, 2, 2, 4],
       [1, 0, 3, 3, 2, 4, 1, 2, 0, 2, 0, 4],
       [2, 0, 2, 3, 2, 2, 1, 2, 2, 2, 2, 4],
       [1, 0, 4, 3, 3, 2, 1, 2, 2, 2, 2, 4],
       [0, 1, 3, 2, 1, 4, 1, 0, 4, 2, 1, 4]])

In [375]:
text ="kick the ball and score the goal"
# text= 'how mana cigarettes does you smoke a daay???'
# text= "I saw a burger then I ate it"
# text  = "share your opinion, shop & search to earn real cash rewards."

verb = 'earn'

attention = get_attention([text], model, tokenizer)

tokens = tokenizer.tokenize(text)
verb_index = tokens.index(verb)+1
item = attention.argmax(-2)[9, :, 3, verb_index].item()

print(tokens[item-1], item-1)

rewards 11
